<a href="https://colab.research.google.com/github/siyeon-Hwang/aiffel_exp/blob/main/%5BNLP-06%5D%20Transformer%20Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 목차
***
Step 1. 데이터 다운로드  
Step 2. 데이터 정제  
Step 3. 데이터 토큰화  
Step 4. Augmentation    
  (1) 모델 불러오기  
(2) Lexical Substitution 구현 및 적용   

Step 5. 데이터 벡터화  
(1) <start><end> 토큰 추가  
(2) 단어사전 구축 및 벡터화  

Step 6. 훈련하기  
(1) Positional Encoding
(2) Mask 생성
(3) Multi-Head Attention
(4) Position-wise Feed Forward Network
(4) Encoder Layer
(5) Decoder Layer
(6) Encoder
(7) Decoder  
(8) Transformer 전체 모델 조립  
(9) 모델 인스턴스 생성  
(10) 모델 학습  
(11) 답변 생성 - error

Step 7. 성능 측정하기 - X

회고

In [1]:
import numpy 
import pandas
import tensorflow
import nltk
import gensim

print(numpy.__version__)
print(pandas.__version__)
print(tensorflow.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.5
1.3.5
2.8.0
3.2.5
3.6.0


konlpy 및 mecab 설치

In [2]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ hash mecab
+ echo 'mecab-ko is already installed'
mecab-ko is already installed
+ [[ -d /usr/local/lib/mecab/dic/mecab-ko-dic ]]
+ echo 'mecab-ko-dic is already installed'
mecab-ko-dic is already installed
++ python3 -c 'import pkgutil; print(1 if pkgutil.find_loader("MeCab") else 0)'
+ [[ 1 == \1 ]]
+ echo 'mecab-python is already installed'
mecab-python is already installed
+ echo Done.
Done.


In [3]:
import numpy as np
import pandas as pd 
import tensorflow as tf 

import re
import os

from konlpy.tag import Mecab

In [4]:
import konlpy
konlpy.tag.Mecab().morphs('아버지가방에들어가신다')


['아버지', '가', '방', '에', '들어가', '신다']

In [5]:
import konlpy
print(konlpy.__version__)

0.6.0


# Step 1. 데이터 다운로드
- [songys/Chatbot_data](https://github.com/songys/Chatbot_data)

위 링크에서 ChatbotData .csv 라는 이름으로 저장된 파일을 다운받고,   
데이터를 읽어와 질문과 답변을 각각 questions, answers 변수에 나눠서 저장한다.

In [6]:
# 1. 경로 바꾸기
#%cd '/content/drive/MyDrive/GoingDeeper_NLP/data/11.transformer_chatbot'

# 2. 데이터 다운로드
#!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv


In [7]:
data = pd.read_csv('/content/drive/MyDrive/GoingDeeper_NLP/data/11.transformer_chatbot/ChatbotData.csv')

data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [8]:
questions = data['Q'].to_string(index=False)
questions = questions.split('\n')
questions[:3]

['                                            12시 땡!',
 '                                       1지망 학교 떨어졌어',
 '                                      3박4일 놀러가고 싶다']

In [9]:
answers = data['A'].to_string(index=False)
answers = answers.split('\n')
answers[:3]

['                                        하루가 또 가네요.',
 '                                         위로해 드립니다.',
 '                                       여행은 언제나 좋죠.']

# Step 2. 데이터 정제
아래 조건을 만족하는 preprocess_sentence() 함수 구현
1.   영문자의 경우, **모두 소문자로 변환**.
2.   영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 **정규식을 활용하여 모두 제거**.

In [10]:
def preprocess_sentence(sentences):
  corpus = []
  for sentence in sentences:
    sentence = sentence.lower().strip()
  
    reg = re.compile('[^a-zㅏ-ㅣㄱ-ㅎ가-힣0-9?.!, ]')
    sentence = reg.sub('', str(sentence))

    sentence = re.sub(r'([?.!,])', r" \1 ", sentence)
    sentence = re.sub(r'([" "]+)', " ", sentence)

    corpus.append(sentence)
  return corpus

In [11]:
questions2 = preprocess_sentence(questions)
questions2[:5]
t = Mecab().morphs
#questions3 = t.morphs(questions2)
#questions3[:3]
#AssertionError: phrase input should be string, not <class 'list'>
lis = []
for s in questions2:
  lis.append(t(s))
lis[:5]

[['12', '시', '땡', '!'],
 ['1', '지망', '학교', '떨어졌', '어'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'],
 ['ppl', '심하', '네']]

# Step 3. 데이터 토큰화
아래 조건을 만족하는 build_corpus() 함수 구현


1.   **소스 문장 데이터**와 **타겟 문장 데이터**를 입력으로 받는다.
2.   데이터를 앞서 정의한 preprocess_sentence() 함수로 **정제하고, 토큰화**
3.   토큰화는 **전달받은 토크나이즈 함수 사용**. 여기서는 mecab.morphs 사용하면 됨
4.   토큰의 개수가 일정 길이 이상인 문장은 **데이터에서 제외**
5.   **중복되는 문장은 데이터에서 제외**. 소스:타겟 쌍을 비교하지 않고 소스는 소스대로, 타겟은 타겟대로 검사. 중복 쌍 흐트러지지 않도록 유의할 것.

구현한 함수를 활용하여 questions와 answers를 각각 que_corpus, ans_corpus 에 토큰화하여 저장한다. 



In [12]:
def build_corpus(src_sentence, tgt_sentence, tokenizer):
  src_sentence = preprocess_sentence(src_sentence)
  tgt_sentence = preprocess_sentence(tgt_sentence)

  src_tokens = []
  for sentence in src_sentence:
    src_tokens.append(tokenizer.morphs(sentence))

  tgt_tokens = []
  for sentence in tgt_sentence:
    tgt_tokens.append(tokenizer.morphs(sentence))

  assert len(src_tokens) == len(tgt_tokens)

  que_corpus = [src_tokens[i] for i in range(len(src_tokens)) if len(src_tokens[i]) <= 50]
  ans_corpus = [tgt_tokens[i] for i in range(len(tgt_tokens)) if len(tgt_tokens[i]) <= 50]

  que_corpus = map(tuple,que_corpus)
  ans_corpus = map(tuple,ans_corpus)
  
  set_corpus = set(zip(que_corpus, ans_corpus))

  que_corpus, ans_corpus = zip(*set_corpus)
  que_corpus = [list(que_corpus[i]) for i in range(len(que_corpus))]
  ans_corpus = [list(ans_corpus[i]) for i in range(len(ans_corpus))]

  return que_corpus, ans_corpus
  

In [13]:
mecab = Mecab()
que_corpus, ans_corpus = build_corpus(questions, answers, mecab)

que_corpus[:5], ans_corpus[:5]

([['가짜', '이별', '4', '개월', '만', '에', '마침표', '찍', '었', '습니다', '.'],
  ['스', '테', '끼', '먹', '은', '거', '몇', '년', '만', '이', '냐'],
  ['좋', '아', '하', '는', '남자', '애', '앞', '에서', '표정', '관리', '가', '안', '돼', '.'],
  ['1000', '일', '만난', '여자', '친구', '와', '이별'],
  ['방금', '너무', '울', '었', '어']],
 [['마침표', '찍', '는', '것', '도', '용기', '가', '필요', '하', '죠', '.'],
  ['맛있', '게', '드세요', '.'],
  ['짝사랑', '도', '이성', '을', '막지', '는', '못하', '나', '봐요', '.'],
  ['더', '오래', '만날', '사람', '만날', '거', '예요', '.'],
  ['속', '이', '조금', '후련', '하', '면', '좋', '겠', '네요', '.']])

# Step 4. Augmentation
주어진 데이터는 1만개 가량으로 적은 편에 속한다.  
**Lexical Substitution**을 적용하여 늘려본다.

아래 링크를 참고하여 **한국어로 사전 훈련된 Embedding 모델을 다운로드**한다.   
Korean (w) 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 Korean(w)를 찾아 다운로드하고, ko.bin 파일을 얻는다.
- [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)

다운로드한 모델을 활용해 **데이터를 Augmentation** 한다. 

*Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 **전체 데이터가 원래의 3배가량으로 늘어나도록** 한다.*

### (1) 모델 불러오기

In [14]:
# 압축 풀기
#%cd '/content/drive/MyDrive/GoingDeeper_NLP/data/12. transformer_chatbot'
#!unzip -qq '/content/drive/MyDrive/GoingDeeper_NLP/data/12. transformer_chatbot/ko.zip'

In [15]:
import gensim

ko_wv = gensim.models.Word2Vec.load('/content/drive/MyDrive/GoingDeeper_NLP/data/12. transformer_chatbot/ko.bin')

ko_wv.wv.most_similar('강아지')

[('고양이', 0.7290452718734741),
 ('거위', 0.7185635566711426),
 ('토끼', 0.7056223154067993),
 ('멧돼지', 0.6950401067733765),
 ('엄마', 0.6934334635734558),
 ('난쟁이', 0.6806551218032837),
 ('한마리', 0.6770296096801758),
 ('아가씨', 0.6750352382659912),
 ('아빠', 0.6729634404182434),
 ('목걸이', 0.6512460708618164)]

### (2) Lexical Substitution 구현 및 적용

In [16]:
# Lexical Substitution 구현하기
import random

def lexical_sub(sentence, word2vec):
  res = []
  toks = sentence

  try:
    _from = random.choice(toks)
    _to = word2vec.wv.most_similar(_from)[0][0]

  except:
    return None

  for tok in toks:
    if tok is _from: 
      res.append(_to)
    else:
      res.append(tok)

  return res

In [17]:
from tqdm.notebook import tqdm

que_corpus_1 = []

for old_src in tqdm(que_corpus):
  new_src = lexical_sub(old_src, ko_wv)
  if new_src is not None:
    que_corpus_1.append(new_src)
  else:
    que_corpus_1.append(old_src)
  
print(que_corpus_1[:3])

  0%|          | 0/11747 [00:00<?, ?it/s]

[['가짜', '이별', '4', '개월', '만', '에', '마침표', '찍', '었', '습니다', '.'], ['스', '타시', '끼', '먹', '은', '거', '몇', '년', '만', '이', '냐'], ['좋', '아', '하', '는', '남자', '사우스캐롤라이', '앞', '에서', '표정', '관리', '가', '안', '돼', '.']]


In [18]:
ans_corpus_1 = []

for old_src in tqdm(ans_corpus):
  new_src = lexical_sub(old_src, ko_wv)
  if new_src is not None:
    ans_corpus_1.append(new_src)
  else:
    ans_corpus_1.append(old_src)
  
print(ans_corpus_1[:3])

  0%|          | 0/11747 [00:00<?, ?it/s]

[['마침표', '찍', '는', '것', '도', '용기', '가', '필요', '하', '지요', '.'], ['맛있', '게', '드세요', '.'], ['짝사랑', '도', '이성', '을', '막지', '는', '않', '나', '봐요', '.']]


In [19]:
que_corpus_new = que_corpus + que_corpus_1 + que_corpus
ans_corpus_new = ans_corpus + ans_corpus + ans_corpus_1

print(len(que_corpus_new))
print(len(ans_corpus_new))

35241
35241


# Step 5. 데이터 벡터화
타겟 데이터인 ans_corpus_new 에 <start\> 토큰과 <end\> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행한다.

### (1) <start\><end\> 토큰 추가

In [20]:
ans_corpus_new1 = []
for sentence in ans_corpus_new:
  ans_corpus_new1.append(["<start>"] + sentence + ["<end>"])

print(len(ans_corpus_new1))
print(ans_corpus_new1[:3])

35241
[['<start>', '마침표', '찍', '는', '것', '도', '용기', '가', '필요', '하', '죠', '.', '<end>'], ['<start>', '맛있', '게', '드세요', '.', '<end>'], ['<start>', '짝사랑', '도', '이성', '을', '막지', '는', '못하', '나', '봐요', '.', '<end>']]


### (2) 단어사전 구축 및 벡터화
ans_corpus_new1 와 que_corpus_new를 결합하여 **전체 데이터에 대한 단어 사전을 구축**하고 **벡터화** 하여 enc_train과 dec_train을 얻는다.

In [21]:
entire_corpus = []
for i in range(len(que_corpus_new)):
  entire_corpus.append(que_corpus_new[i] + ans_corpus_new1[i])

entire_corpus[:3]

[['가짜',
  '이별',
  '4',
  '개월',
  '만',
  '에',
  '마침표',
  '찍',
  '었',
  '습니다',
  '.',
  '<start>',
  '마침표',
  '찍',
  '는',
  '것',
  '도',
  '용기',
  '가',
  '필요',
  '하',
  '죠',
  '.',
  '<end>'],
 ['스',
  '테',
  '끼',
  '먹',
  '은',
  '거',
  '몇',
  '년',
  '만',
  '이',
  '냐',
  '<start>',
  '맛있',
  '게',
  '드세요',
  '.',
  '<end>'],
 ['좋',
  '아',
  '하',
  '는',
  '남자',
  '애',
  '앞',
  '에서',
  '표정',
  '관리',
  '가',
  '안',
  '돼',
  '.',
  '<start>',
  '짝사랑',
  '도',
  '이성',
  '을',
  '막지',
  '는',
  '못하',
  '나',
  '봐요',
  '.',
  '<end>']]

In [22]:
# que_corpus_new 와 ans_corpus_new1 결합
#entire_corpus = list(zip(que_corpus_new, ans_corpus_new1))
#entire_corpus[:3]

In [23]:
def tokenize(corpus, que_corpus, ans_corpus):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000,
                                                    filters='')
  tokenizer.fit_on_texts(corpus)

  tensor_q = tokenizer.texts_to_sequences(que_corpus)
  tensor_a = tokenizer.texts_to_sequences(ans_corpus)

  tensor_q = tf.keras.preprocessing.sequence.pad_sequences(tensor_q,
                                                           padding='post')
  tensor_a = tf.keras.preprocessing.sequence.pad_sequences(tensor_a,
                                                           padding='post')
  
  return tensor_q, tensor_a, tokenizer

In [24]:
enc_train, dec_train, tokenizer = tokenize(entire_corpus, que_corpus_new, ans_corpus_new1)

print('Vocab Size:', len(tokenizer.index_word))
print('enc_train Length: ', len(enc_train))
print('dec_train Length: ', len(dec_train))

Vocab Size: 7937
enc_train Length:  35241
dec_train Length:  35241


In [25]:
enc_train[0], dec_train[0]

(array([1384,   66,  546,  345,   58,   25, 1752,  611,   63,  106,    1,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], dtype=int32),
 array([   2, 1752,  611,    6,   41,   22,  374,    8,  121,    5,   35,
           1,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       dtype=int32))

In [26]:
VOCAB_SIZE = len(tokenizer.index_word)

BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).batch(batch_size=BATCH_SIZE)

# Step 6. 훈련하기
Transformer를 이용해 훈련한다.   
데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있다.  

### (1) Positional Encoding

In [27]:
def positional_encoding(pos, d_model):
  def cal_angle(position, i):
    return position / np.power(10000, (2*(1//2)) / np.float32(d_model))

  def get_posi_angle_vec(position):
    return [cal_angle(position, i) for i in range(d_model)]

  sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

  sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
  sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

  return sinusoid_table

### (2) Mask 생성

In [28]:
def generate_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask

def generate_masks(src, tgt):
  enc_mask = generate_padding_mask(src)
  dec_enc_mask = generate_padding_mask(src)

  dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
  dec_tgt_padding_mask = generate_padding_mask(tgt)
  dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

  return enc_mask, dec_enc_mask, dec_mask

### (3) Multi-Head Attention

In [29]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    self.depth = d_model // self.num_heads

    self.W_q = tf.keras.layers.Dense(d_model)
    self.W_k = tf.keras.layers.Dense(d_model)
    self.W_v = tf.keras.layers.Dense(d_model)

    self.linear = tf.keras.layers.Dense(d_model)

  def scaled_dot_product_attention(self, Q, K, V, mask):
    d_k = tf.cast(K.shape[-1], tf.float32)
    QK = tf.matmul(Q, K, transpose_b=True)

    scaled_qk = QK / tf.math.sqrt(d_k)

    if mask is not None: scaled_qk += (mask * -1e9)

    attentions = tf.nn.softmax(scaled_qk, axis=-1)
    out = tf.matmul(attentions, V)

    return out, attentions

  def split_heads(self, x):
    bsz = x.shape[0]
    split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
    split_x = tf.transpose(split_x, perm=[0,2,1,3])

    return split_x

  def combine_heads(self, x):
    bsz = x.shape[0]
    combined_x = tf.transpose(x, perm=[0,2,1,3])
    combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

    return combined_x

  def call(self, Q, K, V, mask):
    WQ = self.W_q(Q)
    WK = self.W_k(K)
    WV = self.W_v(V)

    WQ_splits = self.split_heads(WQ)
    WK_splits = self.split_heads(WK)
    WV_splits = self.split_heads(WV)

    out, attention_weights = self.scaled_dot_product_attention(
        WQ_splits, WK_splits, WV_splits, mask
    )

    out = self.combine_heads(out)
    out = self.linear(out)

    return out, attention_weights

### (4) Position-wise Feed Forward Network

In [30]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
  def __init__(self, d_model, d_ff):
    super(PoswiseFeedForwardNet, self).__init__()
    self.d_model = d_model
    self.d_ff = d_ff

    self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
    self.fc2 = tf.keras.layers.Dense(d_model)

  def call(self, x):
    out = self.fc1(x)
    out = self.fc2(out)

    return out

### (4) Encoder Layer

In [31]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, n_heads, d_ff, dropout):
    super(EncoderLayer, self).__init__()

    self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
    self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

    self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.do = tf.keras.layers.Dropout(dropout)

  def call(self, x, mask):
    '''
    Multi-Head Attention
    '''
    residual = x
    out = self.norm_1(x)
    out, enc_attn = self.enc_self_attn(out, out, out, mask)
    out = self.do(out)
    out += residual

    '''
    Position-Wise Feed Forward network
    '''
    residual = out
    out = self.norm_2(out)
    out = self.ffn(out)
    out = self.do(out)
    out += residual

    return out, enc_attn

### (5) Decoder Layer

In [32]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, d_ff, dropout):
    super(DecoderLayer, self).__init__()

    self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
    self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

    self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

    self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.do = tf.keras.layers.Dropout(dropout)

  def call(self, x, enc_out, dec_enc_mask, padding_mask):
    '''
    Masked Multi-Head Attention
    '''
    residual = x
    out = self.norm_1(x)
    out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
    out = self.do(out)
    out += residual

    '''
    Multi-Head Attention
    '''
    residual = out
    out = self.norm_2(out)
    # Q, K, V 순서에 주의하세요!
    out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
    out = self.do(out)
    out += residual

    '''
    Position-Wise Feed Forward Network
    '''
    residual = out
    out = self.norm_3(x)
    out = self.ffn(out)
    out = self.do(out)
    out += residual

    return out, dec_attn, dec_enc_attn

### (6) Encoder

In [33]:
class Encoder(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               dropout):
    super(Encoder, self).__init__()
    self.n_layers = n_layers
    self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]

    self.do = tf.keras.layers.Dropout(dropout)

  def call(self, x, mask):
    out = x

    enc_attns = list()
    for i in range(self.n_layers):
      out, enc_attn = self.enc_layers[i](out, mask)
      enc_attns.append(enc_attn)

    return out, enc_attns

### (7) Decoder

In [34]:
class Decoder(tf.keras.Model):
  def __init__(self, 
               n_layers,
               d_model,
               n_heads,
               d_ff,
               dropout):
    super(Decoder, self).__init__()
    self.n_layers = n_layers
    self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]

  def call(self, x, enc_out, dec_enc_mask, padding_mask):
    out = x

    dec_attns = list()
    dec_enc_attns = list()
    for i in range(self.n_layers):
      out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

      dec_attns.append(dec_attn)
      dec_enc_attns.append(dec_enc_attn)

    return out, dec_attns, dec_enc_attns

### (8) Transformer 전체 모델 조립

In [35]:
class Transformer(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               src_vocab_size,
               tgt_vocab_size,
               pos_len,
               dropout=0.2,
               shared_fc=True,
               shared_emb=True):
    super(Transformer, self).__init__()

    self.d_model = tf.cast(d_model, tf.float32)

    if shared_emb:
      self.enc_emb = self.dec_emb = \
      tf.keras.layers.Embedding(src_vocab_size, d_model)
    else:
      self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
      self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

    self.pos_encoding = positional_encoding(pos_len, d_model)
    self.do = tf.keras.layers.Dropout(dropout)

    self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
    self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

    self.fc = tf.keras.layers.Dense(tgt_vocab_size)

    self.shared_fc = shared_fc

    if shared_fc:
      self.fc.set_weights(tf.transpose(self.dec_emb.weights))

  def embedding(self, emb, x):
    seq_len = x.shape[1]

    out = emb(x)

    if self.shared_fc: out *= tf.math.sqrt(self.d_model)

    out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
    out = self.do(out)

    return out

  def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
    enc_in = self.embedding(self.enc_emb, enc_in)
    dec_in = self.embedding(self.dec_emb, dec_in)

    enc_out, enc_attns = self.encoder(enc_in, enc_mask)

    dec_out, dec_attns, dec_enc_attns = \
    self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)

    logits = self.fc(dec_out)

    return logits, enc_attns, dec_attns, dec_enc_attns

### (9) 모델 인스턴스 생성

In [42]:
SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = VOCAB_SIZE

transformer = Transformer(n_layers = 2,
                          d_model = 368,
                          n_heads = 8,
                          d_ff = 1024,
                          src_vocab_size = SRC_VOCAB_SIZE,
                          tgt_vocab_size=TGT_VOCAB_SIZE,
                          pos_len = 50,
                          dropout=0.2,
                          shared_fc = True,
                          shared_emb=True)

### (10) 모델 학습


#### 1) Learning Rate Scheduler

In [43]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=1000):
    super(LearningRateScheduler, self).__init__()

    self.d_model = d_model
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = step ** -0.5
    arg2 = step * (self.warmup_steps ** -1.5)

    return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

#### 2) Learning Rate & Optimizer

In [44]:
d_model = 368

learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1 = 0.9,
                                     beta_2 = 0.98,
                                     epsilon = 1e-6)

#### 3) Loss Function

In [45]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction = 'none'
)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real,0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

#### 4) Train Step 정의

In [46]:
@tf.function()
def train_step(src, tgt, model, optimizer):
  tgt_in = tgt[:, :-1]  # Decoder의 input
  gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

  enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

  with tf.GradientTape() as tape:
    predictions, enc_attns, dec_attns, dec_enc_attns = \
    model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)

    loss = loss_function(gold, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return loss, enc_attns, dec_attns, dec_enc_attns

#### 5) 훈련

In [47]:
EPOCHS = 10

for epoch in range(EPOCHS):
  total_loss = 0

  dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
  tqdm_bar = tqdm(total=dataset_count)
  for step, (enc_batch, dec_batch) in enumerate(train_dataset):
    batch_loss, enc_attns, dec_attns, dec_enc_attns = \
    train_step(enc_batch,
               dec_batch,
               transformer,
               optimizer)
    
    total_loss += batch_loss

    tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
    tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (step + 1)))
    tqdm_bar.update()

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

  0%|          | 0/551 [00:00<?, ?it/s]

### (11) 답변 생성

In [63]:
tokenizer.texts_to_sequences(['<start>'])[0][0]

2

In [73]:
# 번역 생성 함수
def evaluate(sentence, model, tokenizer):
  sentence = preprocess_sentence(sentence)

  tokens = tokenizer.texts_to_sequences(sentence)

  _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                         maxlen=enc_train.shape[-1],
                                                         padding = 'post')
  ids = []
  output = tf.expand_dims([tokenizer.text_to_sequence(['<start>'])[0][0]], 0)
  for i in range(dec_train.shape[-1]):
    enc_padding_mask, combined_mask, dec_padding_mask = \
    generate_masks(_input, output)

    predictions, enc_attns, dec_attns, dec_enc_attns = \
    model(_input,
          output,
          enc_padding_mask,
          combined_mask,
          dec_padding_mask)
    
    predicted_id = \
    tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

    if tokenizer.texts_to_sequences(['<end>'])[0][0] == predicted_id:
      result = tokenizer.sequences_to_texts(ids)
      return sentence, result, enc_attns, dec_attns, dec_enc_attns

    ids.append(predicted_id)
    output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

  result = tokenizer.sequences_to_texts(ids)

  return sentence, result, enc_attns, dec_attns, dec_enc_attns

In [74]:
# 번역 생성 및 Attention 시각화 결합
def translate(sentence, model, tokenizer):
  sentence, result, enc_attns, dec_attns, dec_enc_attns = \
  evaluate(sentence, model, tokenizer)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

In [75]:
examples = ['지루하다, 놀러가고 싶어.',
            '오늘 일찍 일어났더니 피곤하다.',
            '간만에 여자친구랑 데이트 하기로 했어.',
            '집에 있는다는 소리야.']

for example in examples:
  translate(example, transformer, tokenizer)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/sequence.py:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sample_shape = np.asarray(x).shape[1:]


ValueError: ignored

# Step 7. 성능 측정하기

***
# 회고
모델을 훈련시키는 부분부터 뭔가 잘못됐음을 알았다. 왜 loss값이 안 나올까..  epoch 1 이 진행되는 동안 loss가 3까지 떨어지다가 epoch 2로 전환되면서 갑자기 nan으로 뜬다.   
왜 이런 건지 더 찾아봐야겠다.

어쨌거나 학습은 됐다고 뜨니,  
다른 예문을 가져와서 답변을 생성해보려 했으나, 위에서 볼 수 있다시피 ValueError가 뜬다.  
문제는 (11) 답변 생성 부분의 코드부터 이해를 못했다는 것이다. 에러내용을 보니까 데이터 차원이 어디서 잘못된 모양인데 어디서부터 잘못된 건지 모르겠다. 
